In [3]:
import numpy as np

class Audiobooks_Data_Reader():
    
    def __init__(self, dataset, batch_size = None):
    
        npz = np.load('Audiobooks_data_{0}.npz'.format(dataset))
        
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    # A method which loads the next batch
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        return inputs_batch, targets_one_hot
    
    def __iter__(self):
        return self

In [4]:
import tensorflow as tf

input_size = 10
output_size = 2
hidden_layer_size = 50

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.int32, [None, output_size])

weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])
outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])
outputs_2 = tf.nn.sigmoid(tf.matmul(outputs_1, weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])
outputs = tf.matmul(outputs_2, weights_3) + biases_3

loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)
mean_loss = tf.reduce_mean(loss)

out_equals_target = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

optimize = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(mean_loss)

sess = tf.InteractiveSession()

initializer = tf.global_variables_initializer()
sess.run(initializer)

batch_size = 1000

max_epochs = 1000
prev_validation_loss = 9999999.

train_data = Audiobooks_Data_Reader('train', batch_size)
validation_data = Audiobooks_Data_Reader('validation')
 
for epoch_counter in range(max_epochs):
    
    curr_epoch_loss = 0.
    
    for input_batch, target_batch in train_data:
        _, batch_loss = sess.run([optimize, mean_loss], 
            feed_dict={inputs: input_batch, targets: target_batch})
        
        curr_epoch_loss += batch_loss
    
    curr_epoch_loss /= train_data.batch_count
    
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
            feed_dict={inputs: input_batch, targets: target_batch})
    
    print('Epoch '+str(epoch_counter+1)+
          '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
          '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    
    if validation_loss > prev_validation_loss:
        break
        
    prev_validation_loss = validation_loss
    
print('End of training.')

Epoch 1. Training loss: 1.394. Validation loss: 1.458. Validation accuracy: 47.20%
Epoch 2. Training loss: 1.383. Validation loss: 1.445. Validation accuracy: 47.20%
Epoch 3. Training loss: 1.371. Validation loss: 1.433. Validation accuracy: 47.20%
Epoch 4. Training loss: 1.360. Validation loss: 1.421. Validation accuracy: 47.20%
Epoch 5. Training loss: 1.349. Validation loss: 1.410. Validation accuracy: 47.20%
Epoch 6. Training loss: 1.338. Validation loss: 1.398. Validation accuracy: 47.20%
Epoch 7. Training loss: 1.326. Validation loss: 1.386. Validation accuracy: 47.20%
Epoch 8. Training loss: 1.315. Validation loss: 1.374. Validation accuracy: 47.20%
Epoch 9. Training loss: 1.304. Validation loss: 1.362. Validation accuracy: 47.20%
Epoch 10. Training loss: 1.293. Validation loss: 1.351. Validation accuracy: 47.20%
Epoch 11. Training loss: 1.282. Validation loss: 1.339. Validation accuracy: 47.20%
Epoch 12. Training loss: 1.272. Validation loss: 1.328. Validation accuracy: 47.20%
E

Epoch 100. Training loss: 0.691. Validation loss: 0.705. Validation accuracy: 54.36%
Epoch 101. Training loss: 0.688. Validation loss: 0.702. Validation accuracy: 54.59%
Epoch 102. Training loss: 0.686. Validation loss: 0.700. Validation accuracy: 55.93%
Epoch 103. Training loss: 0.684. Validation loss: 0.697. Validation accuracy: 56.60%
Epoch 104. Training loss: 0.681. Validation loss: 0.695. Validation accuracy: 56.60%
Epoch 105. Training loss: 0.679. Validation loss: 0.692. Validation accuracy: 56.82%
Epoch 106. Training loss: 0.677. Validation loss: 0.690. Validation accuracy: 56.82%
Epoch 107. Training loss: 0.675. Validation loss: 0.688. Validation accuracy: 56.82%
Epoch 108. Training loss: 0.673. Validation loss: 0.685. Validation accuracy: 56.82%
Epoch 109. Training loss: 0.671. Validation loss: 0.683. Validation accuracy: 57.27%
Epoch 110. Training loss: 0.669. Validation loss: 0.681. Validation accuracy: 57.49%
Epoch 111. Training loss: 0.667. Validation loss: 0.679. Validati

Epoch 199. Training loss: 0.582. Validation loss: 0.590. Validation accuracy: 67.11%
Epoch 200. Training loss: 0.581. Validation loss: 0.589. Validation accuracy: 66.89%
Epoch 201. Training loss: 0.581. Validation loss: 0.589. Validation accuracy: 67.11%
Epoch 202. Training loss: 0.580. Validation loss: 0.588. Validation accuracy: 67.11%
Epoch 203. Training loss: 0.579. Validation loss: 0.587. Validation accuracy: 67.11%
Epoch 204. Training loss: 0.579. Validation loss: 0.587. Validation accuracy: 67.11%
Epoch 205. Training loss: 0.578. Validation loss: 0.586. Validation accuracy: 67.34%
Epoch 206. Training loss: 0.577. Validation loss: 0.585. Validation accuracy: 67.34%
Epoch 207. Training loss: 0.577. Validation loss: 0.585. Validation accuracy: 67.11%
Epoch 208. Training loss: 0.576. Validation loss: 0.584. Validation accuracy: 67.11%
Epoch 209. Training loss: 0.575. Validation loss: 0.583. Validation accuracy: 67.34%
Epoch 210. Training loss: 0.575. Validation loss: 0.583. Validati

Epoch 296. Training loss: 0.522. Validation loss: 0.534. Validation accuracy: 69.13%
Epoch 297. Training loss: 0.522. Validation loss: 0.534. Validation accuracy: 69.13%
Epoch 298. Training loss: 0.521. Validation loss: 0.533. Validation accuracy: 69.13%
Epoch 299. Training loss: 0.521. Validation loss: 0.533. Validation accuracy: 68.90%
Epoch 300. Training loss: 0.520. Validation loss: 0.532. Validation accuracy: 68.90%
Epoch 301. Training loss: 0.520. Validation loss: 0.532. Validation accuracy: 68.90%
Epoch 302. Training loss: 0.519. Validation loss: 0.531. Validation accuracy: 68.90%
Epoch 303. Training loss: 0.518. Validation loss: 0.531. Validation accuracy: 68.90%
Epoch 304. Training loss: 0.518. Validation loss: 0.530. Validation accuracy: 68.90%
Epoch 305. Training loss: 0.517. Validation loss: 0.530. Validation accuracy: 68.90%
Epoch 306. Training loss: 0.517. Validation loss: 0.529. Validation accuracy: 68.90%
Epoch 307. Training loss: 0.516. Validation loss: 0.529. Validati

Epoch 394. Training loss: 0.477. Validation loss: 0.493. Validation accuracy: 69.80%
Epoch 395. Training loss: 0.476. Validation loss: 0.492. Validation accuracy: 70.25%
Epoch 396. Training loss: 0.476. Validation loss: 0.492. Validation accuracy: 70.25%
Epoch 397. Training loss: 0.476. Validation loss: 0.491. Validation accuracy: 70.25%
Epoch 398. Training loss: 0.475. Validation loss: 0.491. Validation accuracy: 70.25%
Epoch 399. Training loss: 0.475. Validation loss: 0.491. Validation accuracy: 70.25%
Epoch 400. Training loss: 0.475. Validation loss: 0.490. Validation accuracy: 70.69%
Epoch 401. Training loss: 0.474. Validation loss: 0.490. Validation accuracy: 70.69%
Epoch 402. Training loss: 0.474. Validation loss: 0.490. Validation accuracy: 70.69%
Epoch 403. Training loss: 0.473. Validation loss: 0.489. Validation accuracy: 70.69%
Epoch 404. Training loss: 0.473. Validation loss: 0.489. Validation accuracy: 70.69%
Epoch 405. Training loss: 0.473. Validation loss: 0.489. Validati

Epoch 494. Training loss: 0.444. Validation loss: 0.463. Validation accuracy: 72.71%
Epoch 495. Training loss: 0.444. Validation loss: 0.463. Validation accuracy: 72.71%
Epoch 496. Training loss: 0.444. Validation loss: 0.463. Validation accuracy: 72.71%
Epoch 497. Training loss: 0.443. Validation loss: 0.462. Validation accuracy: 72.71%
Epoch 498. Training loss: 0.443. Validation loss: 0.462. Validation accuracy: 72.71%
Epoch 499. Training loss: 0.443. Validation loss: 0.462. Validation accuracy: 72.71%
Epoch 500. Training loss: 0.443. Validation loss: 0.462. Validation accuracy: 72.71%
Epoch 501. Training loss: 0.442. Validation loss: 0.461. Validation accuracy: 72.71%
Epoch 502. Training loss: 0.442. Validation loss: 0.461. Validation accuracy: 72.71%
Epoch 503. Training loss: 0.442. Validation loss: 0.461. Validation accuracy: 72.71%
Epoch 504. Training loss: 0.441. Validation loss: 0.461. Validation accuracy: 72.93%
Epoch 505. Training loss: 0.441. Validation loss: 0.460. Validati

Epoch 592. Training loss: 0.421. Validation loss: 0.442. Validation accuracy: 74.94%
Epoch 593. Training loss: 0.421. Validation loss: 0.442. Validation accuracy: 74.94%
Epoch 594. Training loss: 0.421. Validation loss: 0.442. Validation accuracy: 74.94%
Epoch 595. Training loss: 0.421. Validation loss: 0.442. Validation accuracy: 74.94%
Epoch 596. Training loss: 0.420. Validation loss: 0.441. Validation accuracy: 74.94%
Epoch 597. Training loss: 0.420. Validation loss: 0.441. Validation accuracy: 74.94%
Epoch 598. Training loss: 0.420. Validation loss: 0.441. Validation accuracy: 74.94%
Epoch 599. Training loss: 0.420. Validation loss: 0.441. Validation accuracy: 74.94%
Epoch 600. Training loss: 0.420. Validation loss: 0.441. Validation accuracy: 74.94%
Epoch 601. Training loss: 0.419. Validation loss: 0.441. Validation accuracy: 74.94%
Epoch 602. Training loss: 0.419. Validation loss: 0.440. Validation accuracy: 74.94%
Epoch 603. Training loss: 0.419. Validation loss: 0.440. Validati

Epoch 700. Training loss: 0.402. Validation loss: 0.425. Validation accuracy: 75.17%
Epoch 701. Training loss: 0.402. Validation loss: 0.425. Validation accuracy: 75.17%
Epoch 702. Training loss: 0.402. Validation loss: 0.425. Validation accuracy: 75.39%
Epoch 703. Training loss: 0.402. Validation loss: 0.425. Validation accuracy: 75.17%
Epoch 704. Training loss: 0.402. Validation loss: 0.425. Validation accuracy: 75.17%
Epoch 705. Training loss: 0.402. Validation loss: 0.425. Validation accuracy: 75.17%
Epoch 706. Training loss: 0.401. Validation loss: 0.425. Validation accuracy: 75.17%
Epoch 707. Training loss: 0.401. Validation loss: 0.424. Validation accuracy: 75.17%
Epoch 708. Training loss: 0.401. Validation loss: 0.424. Validation accuracy: 75.17%
Epoch 709. Training loss: 0.401. Validation loss: 0.424. Validation accuracy: 75.17%
Epoch 710. Training loss: 0.401. Validation loss: 0.424. Validation accuracy: 75.17%
Epoch 711. Training loss: 0.401. Validation loss: 0.424. Validati

Epoch 814. Training loss: 0.387. Validation loss: 0.412. Validation accuracy: 75.39%
Epoch 815. Training loss: 0.387. Validation loss: 0.411. Validation accuracy: 75.39%
Epoch 816. Training loss: 0.387. Validation loss: 0.411. Validation accuracy: 75.39%
Epoch 817. Training loss: 0.387. Validation loss: 0.411. Validation accuracy: 75.39%
Epoch 818. Training loss: 0.387. Validation loss: 0.411. Validation accuracy: 75.39%
Epoch 819. Training loss: 0.387. Validation loss: 0.411. Validation accuracy: 75.39%
Epoch 820. Training loss: 0.387. Validation loss: 0.411. Validation accuracy: 75.39%
Epoch 821. Training loss: 0.386. Validation loss: 0.411. Validation accuracy: 75.39%
Epoch 822. Training loss: 0.386. Validation loss: 0.411. Validation accuracy: 75.39%
Epoch 823. Training loss: 0.386. Validation loss: 0.411. Validation accuracy: 75.39%
Epoch 824. Training loss: 0.386. Validation loss: 0.410. Validation accuracy: 75.39%
Epoch 825. Training loss: 0.386. Validation loss: 0.410. Validati

Epoch 915. Training loss: 0.376. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 916. Training loss: 0.376. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 917. Training loss: 0.376. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 918. Training loss: 0.376. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 919. Training loss: 0.376. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 920. Training loss: 0.376. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 921. Training loss: 0.375. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 922. Training loss: 0.375. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 923. Training loss: 0.375. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 924. Training loss: 0.375. Validation loss: 0.401. Validation accuracy: 76.29%
Epoch 925. Training loss: 0.375. Validation loss: 0.400. Validation accuracy: 76.29%
Epoch 926. Training loss: 0.375. Validation loss: 0.400. Validati